# 1 . Codes de Hamming (Supprimés de ce fichier)

<a id="solomon"></a>
# 2.3 Les codes de Reed-Solomon

<a id="solomon1"></a>
## 2.3.1. Introduction aux codes cycliques
Les codes présentés précedemment faisaient jusqu'à lors partie des [codes linéaires](https://fr.wikipedia.org/wiki/Code_lin%C3%A9aire). Il existe cependant une famille de codes issue de ces codes linéaires s'appelant les [codes cycliques](https://fr.wikipedia.org/wiki/Code_cyclique). 

Cette notion sera expliquée ci-dessous, mais il faut pour l'instant simplement savoir qu'un code linéaire existe dans un corps fini, avec $p$ valeurs possibles pour chaque élément. On note alors ces champs $GF(p)$, les codes présentés ci-dessus faisaient parties du $GF(2)$, avec comme valeurs possibles {0,1}, c.à.d des bits.

On appelle un code cyclique un code qui, lorsqu'il appartient au corps fini $GF(p^m)$, et est soumis à un [déplacement vers la droite](https://en.wikipedia.org/wiki/Circular_shift) de tous ses bits, donne un nouveau code valide.
![illustration_codes_cycliques.png](https://upload.wikimedia.org/wikipedia/commons/thumb/3/37/Rotate_right.svg/300px-Rotate_right.svg.png)


Cette famille de dite de **Reed-Solomon** est très puissante et utilisée dans de nombreux domaines, notamment pour pouvoir corriger les rayures des CD, DVD, ou bien pour faciliter la prise en photo d'un QR Code. Elle encode un message en blocs de données de taille $n = 2^m$, avec $k$ symboles (notre symbole sera un octet) d'information. avec $k<n$ on a alors $n-k$ symboles qui serviront à détecter les erreurs.

On aura alors un code $C$ tel que (cf. [Wikipédia](https://en.wikipedia.org/wiki/Cyclic_code)): 
$$
\mathbf {C} ={\Big \{}\;{\big (}p(a_{1}),p(a_{2}),\dots ,p(a_{n}){\big )}\;{\Big |}\;p{\text{ est un polynôme de }}GF(p^m){\text{ de degré }}<k\;{\Big \}}\,.
$$
<a id="solomon2"></a>
## 2.3.2. Les champs de Galois

La famille de code dite de **Reed-Solomon** est une famille de code cyclique basée sur les corps finis (aussi appelés les champs de Galois, qui seront abbreviés GF pour Galois Field en anglais).


Ces champs de Galois ont une notation $GF(p^n)$.
Un $GF(p)$ avec $p$ un nombre premier représente [l'anneau](https://en.wikipedia.org/wiki/Ring_(mathematics)) d'entiers modulo $p$

Les éléments de $GF(p^n)$ peuvent être représentés par des polynomes de degrés strictement inférieurs à $n$ dans le groupe $GF(p)$. On effectue alors des opérations modulo $g(x)$, avec $g(x)$ un polynome irréductible de degré $n$ dans $GF(p)$ dit **générateur**

Il est possible dans Sagemath de créer des $GF$ relativement facilement. Cependant, ils ne seront pas utilisés par la suite car il nous semblait plus simple d'en faire notre propre implémentation, nous permettant de mieux nous approprier le sujet et de le comprendre.

On utilisera ici un $GF(2 ^8)$, signifiant que chaque symbole (ici un caractère, codé sur un octet), sera représenté par $n=8$ éléments pouvant prendre $p=2$ valeurs (1 ou 0), c.à.d. 8 bits. Le $GF(2^8)$ est donc tout naturellement un très bon choix pour traiter des chaînes de caractères

In [2]:
champ = GF(2^8,'alpha')
champ

Finite Field in alpha of size 2^8

On voit notre variable champ est un $GF(2^8)$ 


Leur éléments peuvent être représentés dans une table, avec pour chaque élément le polynome associé :

In [5]:
#Executez ce code pour voir la table des racines. La table peut prendre du temps à s'afficher
table(
      [("Numero d'élément dans la table des racines", "Racine associée")]+
      [(i,x) for i,x in list(enumerate(champ))[:16]],
      header_row=True
     )

Numero d'élément dans la table des racines,Racine associée
,
,
,
,
,
,
,
,
,
,


On voit donc dans cette table qu'au lieu d'arriver à $\alpha^8$ à l'indice 8, on a un polynome de la forme $\alpha^4 + \alpha^3 +\alpha^2 +1$

Cela est dû à la forme du polynôme générateur $g(x) = x^8 + x^4 +x^3 +x^2 +1$ utilisé par Sagemath pour former le $GF(2^8)$

$\alpha$ étant une racine du $GF(2^8)$, on peut écrire 
<br>
    $p(\alpha) = 0$
<br>
    $\alpha^8 + \alpha^4 + \alpha^3 +\alpha^2 +1 = 0$
<br>
    $-\alpha^8 = \alpha^4 + \alpha^3 +\alpha^2 +1$
<br>
    $\alpha^8 = \alpha^4 + \alpha^3 +\alpha^2 +1$, avec $\alpha^8 = -\alpha^8 [2]$
<br>

D'où la forme de l'indice numéro 8 dans le $GF(2^8)$

Si ces notions semblent floues, elles seront éclairées plus tard avec une notation en binaire pour les polynomes, et l'apparition de table dites **logarithmiques** et **antilogarithmiques** dans lesquelles ont gardera une trace de la notation en binaire des polynomes

<a id="solomon3"></a>
## 2.3.3. L'implémentation des codes de Reed-Solomon en Python

Pour mieux comprendre les codes de Reed-Solomon ainsi que les champs de Galois, nous avons décidé, aidés de plusieurs sources citées en fin de document, d'implémenter nous même les espaces de Galois. Cela nous permet aussi d'implémenter des fonctions plus complexes, telles que des [divisions polynomiales](https://en.wikipedia.org/wiki/Synthetic_division).
<a id="solomon31"></a>
### 2.3.3.1. Implémentation des fonctions et classes essentielles
**Commençons par implémenter notre classe ReedSolomon :**

In [1]:
class ReedSolomon:   
    # On travaille dans un corps de Galois de valeur 2^8 = 256
    gfSize = 256 
    # http://icaiit.org/proceedings/2nd_ICAIIT/7.pdf
    # Le polynome générateur de la forme x^8 + x^4 + x^3 + x^2 + 1 = 100011101 = 285
    genPoly = 285 
    log = [0]*gfSize
    antilog = [0]*gfSize
    
    def _trimArray(self,arr):
        if len(arr)>0:
            while(len(arr)>0 and arr[0]==0): arr.pop(0)

On  voit que cette classe possède plusieurs attributs : D'abord **gfSize** la taille du $GF$ : 256, pour $GF(2^8)$

Ensuite, on a **genPoly**, le polynome générateur, $x^8+x^4+x^3+x^2+1$, qui sera noté 285, pour sa représentation en binaire $100011101$ avec le bit le plus à gauche représentant le plus grand degré. Les coefficients $a_n$ sont tous compris entre 1 et 0 de part leur appartenant au champs $GF(2^8)$, d'ou la possibilité de représenter ces polynômes sous une forme binaire

Nous pouvons ensuite remarquer 2 variables, **log** et **antilog**,
La table **antilog** représente les éléments du champs, de 1 à 256.
La table **log** représente l'indice des valeurs de la table antilog, ces deux tables sont inversées : 

$log[1] = 0$ => la valeur 1 est le premier élément du $GF(2^8)$<br>
$antilog[0] = 1$ => la première valeur du $GF(2^8)$ est 1.<br>
Ces tables seront particulièrement utiles pour effectuer des multiplications.

La fonction **\_trimArray()** servira simplement à enlever les 0 parasites à gauche des polynômes

Nous allons donc apprendre à créer ces tables, en implémentant la fonction **\_genLogAntilogArrays()**

Le pseudo code sera comme tel : 
```
DEBUT : 
fonction _genLogAntilogArrays 
    initialiser antilog[0] à 1
    initialiser antilog[255] à 1
    pour i allant de 1 à 254 inclus
        antilog[i] = 2*antilog[i-1] (on déplace tout les bits de une position vers la gauche)
        si antilog[i] est supérieur ou égal à 256
            antilog[i] -= polynomeGenerateur
        log[antilog[i]] = i
FIN
```
Et voici l'implémentation en python de cet algorithme,

In [ ]:
# On surcharge la classe ReedSolomon en héritant d'elle meme simplement pour pouvoir 
# découper la classe en plusieurs cellules
class ReedSolomon(ReedSolomon): 
    def _genLogAntilogArrays(self):
        self.antilog[0] = 1
        #log[0] n'existe pas
        self.log[0] = None
        #antilog[0] = antilog[255] car on retourne au début du cycle
        self.antilog[255] = 1
        for i in range(1,255):
            #On multiplie par 2 (équivalent à déplacer tout les bits d'une position vers la gauche, et plus parlant comme on utilise un code cyclique)
            self.antilog[i] = self.antilog[i-1] << 1 
            if self.antilog[i] >= self.gfSize:
                self.antilog[i] = self.antilog[i] ^^ self.genPoly #Operateur XOR sous sagemath
            self.log[self.antilog[i]] = i
            
    def __init__(self):
        self._genLogAntilogArrays()

Si vous avez été attentif, vous aurez remarqué plusieurs éléments perturbants : 

• Dans le code en Python, on n'effectue pas une soustraction, mais un opérateur XOR entre **antilog et genPoly**

• Dans le pseudocode, on commence à $antilog[0] = 1$, ce qui signifie que $antilog[8] = 256$ car on multiplie toujours par 2. Ce qui signifie que l'on aura $256-285 = -29$ ! 


Pour la première question, cela peut etre expliqué 

Pour expliquer cela, il faut alors comprendre comment fonctionne les additions dans $GF(2^8)$ : 

**Voici la table de vérité d'une addition dans $GF(2^8)$**
<table style="text-align:center; font-size:18px">
<tbody><tr>
<th>+</th>
<th>0</th>
<th>1
</th></tr>
<tr>
<th>0
</th>
<td>0</td>
<td>1
</td></tr>
<tr>
<th>1
</th>
<td>1</td>
<td>0
</td></tr></tbody></table>

On peut donc facilement voir que la table des additions s'apparente à une table de vérité d'un XOR binaire

Pour la deuxième question, on peut voir ça sous plusieurs angle : 

1. La soustraction est dans cet espace égale à l'addition, car $a\oplus a = 0$, on a alors $256 + 285 = 541 \equiv 29(\textrm{mod}\ 256)$

2. Comme l'addition est un XOR, on fait simplement $256\oplus 285 = 29$

In [ ]:
rs = ReedSolomon()
rs._genLogAntilogArrays()
print("Antilog :",rs.antilog[:16])

print("Antilog binaire :",list(map(lambda x:bin(x)[2:],rs.antilog[:16])))

On peut donc voir que l'indice 7 (8e élément) devient effectivement 29.

Si on étudie la structure binaire, on peut voir dans l'ordre 

1, $\alpha, \alpha^2, \alpha^3, \alpha^4, \alpha^5, \alpha^6, \alpha^7, (\alpha^4+\alpha^3+\alpha^2+1$).

Ces tables sont donc correctement crées.

Nous allons donc créer les différentes fonctions et opérations nécessaires à l'encodage du polynome d'informations.
<a id="solomon311"></a>
#### 2.3.3.1.1 La multiplication

Nous avons créé nos tables logarithmiques et antilogarithmiques.
En utilisant les propriété du logarithme, on sait que : $$log(a)+log(b) = log(a*b)$$
On peut donc facilement effectuer des multiplications en utilisant cette propriété : 


In [ ]:
class ReedSolomon(ReedSolomon): 
    def _galMult(self,x,y):
            if ((x == 0) or (y == 0)):
                val = 0
            else:
                val = self.antilog[(self.log[x] + self.log[y])%255]
            return val

<a id="solomon312"></a>
#### 2.3.3.1.2 La division

L'inverse d'une multiplication, encore en utilisant les propriétés du logarithme, tout en restant modulo 255 :$$log(a)-log(b) = log(a/b)$$

In [ ]:
class ReedSolomon(ReedSolomon): 
    def _galDiv(self, x, y):
        val = 0
        if (y == 0):
            raise ZeroDivisionError()
        if (x == 0):
            val = 0
        else:
            val = self.antilog[(self.log[x] - self.log[y] + 255)%255]
        return val

<a id="solomon313"></a>
#### 2.3.3.1.3 La fonction puissance

Il sera par la suite utile d'avoir cette fonction lorsque nous rechercherons des racines d'un polynôme grâce à [l'algorithme de recherche de Chien](https://en.wikipedia.org/wiki/Chien_search)

On cette propriété du logarithme : $$log(a^n) = nlog(a)$$

In [ ]:
class ReedSolomon(ReedSolomon): 
    def _galPow(self,x, p):
        if x == 0:
            return 0
        return self.antilog[(self.log[x] * p) % 255] #On reste congru modulo 256 (donc indice 255)

<a id="solomon314"></a>
#### 2.3.3.1.4 L'addition polynomiale

On ajoute cette fonction basique mais utile, il suffit d'ajouter un a un les coefficients des polynômes :

In [ ]:
class ReedSolomon(ReedSolomon): 
    def _galPolynomialAddition(self, a, b):
            polSum = [0] * max(len(a), len(b))
            for index in range(0, len(a)):
                polSum[index + len(polSum) - len(a)] = a[index]
            for index in range(0, len(b)):
                polSum[index + len(polSum) - len(b)] ^^= b[index]
            return (polSum)

<a id="solomon315"></a>
#### 2.3.3.1.5. La multiplication polynomiale

Comme nous traitons ici des polynomes, il nous faut la possibilité d'effectuer des multiplications polynomiales. Pour cela, il suffit de multiplier chaque élément d'un polynome A par tout les élements d'un polynome B



In [ ]:
class ReedSolomon(ReedSolomon): 
    def _galMultiplicationPolynomiale(self, x,y):
            result = [0]*(len(x)+len(y)-1)
            for i in range(len(x)):
                for j in range(len(y)):
                    result[i+j] ^^= self._galMult(x[i],y[j])
            return result

<a id="solomon316"></a>
#### 2.3.3.1.6 La division polynomiale

La division polynomiale est plus complexe. Nous utiliserons donc cette [implémentation](https://rosettacode.org/wiki/Polynomial_synthetic_division) d'une [division synthétique](https://en.wikipedia.org/wiki/Synthetic_division), adaptée pour le $GF(2^8) permettant d'effectuer une division polynomiale très efficacement

In [ ]:
class ReedSolomon(ReedSolomon): 
    def _galPolynomialDivision(self,dividend, divisor):
        sor = divisor.copy()
        end = dividend.copy()
        self._trimArray(end)
        self._trimArray(sor)
        normalizer = sor[0]
        if(len(end)==1 or len(sor)==1):
            end.append(0)
            sor.append(0)
        out = end.copy()
        for i in range(len(end)-(len(sor)-1)):
            out[i] =self._galDiv(out[i],normalizer)
            coef = out[i]
            if coef != 0: 
                for j in range(1, len(sor)): 
                    out[i + j] ^^= self._galMult(sor[j], coef)
        separator = -(len(sor)-1)
        return out[:separator], out[separator:]

#### 2.3.3.1.7 Le développement d'un polynôme

Nous devons donc créer une fonction qui va développer notre polynome générateur, actuellement stocké sous la forme :$$(x+\alpha^0)(x+\alpha^1)(x+\alpha^2)(x+\alpha^3)...(x+\alpha^n)...(x+\alpha^{2t-1})$$ avec chaque terme $log[n] = \alpha^n$, c.à.d $(x+\alpha^n) = (x+log[n])$

In [ ]:
class ReedSolomon(ReedSolomon): 
    def _galDvtPolynome(self,nbErrorBits):
        # On commence à multiplier à la valeur 1
        result = [1]
        # On multiplie autant de racines que de bits nécessaires pour trouver les erreurs => (x + 1)*(x + 2)*(x + 4)*....*(x +a**errorBits)
        for index in range(nbErrorBits):
            # Racine du polynome (x + a**index)
            root = [1, self.antilog[index]]
            result = self._galMultiplicationPolynomiale(result, root)
        return result

On peut expliquer facilement cet algorithme en expliquant les premières lignes : 
$res = 1$<br>
$res = 1 * (x + 1) = (x+1)$<br>
$res = (x+1) * (x + 2) = x^2+3x+2$<br>
$res = x^2+3x+2 *(x+4) = x^4+7x^2+14x+8$<br>

Avec root égal à $(x+log[n]) = (x+\alpha^n)$

<a id="solomon32"></a>
### 2.3.3.2 Encodage d'un message

Nous avons désormais assez d'outils pour passer à **l'encodage** d'un message envoyé.

L'encodage se déroulera de cette manière : 

```
DEBUT
fonction encode
    polynome = appel _galDvtPolynome()
    listeCaractèresNombres = caractères_vers_nombre(listeCaractères)
    ajouter à listeCaractèresNombres le reste listeCaractèresNombres/polynome
    renvoyer listeCaractèresNombres
FIN
```

Ce message encodé aura alors la forme 
$$M(x) = I(x)x^{2t} + r(x)$$
avec $M(x)$ le message encodé, $I(x)$ le polynome d'information, $r(x)$ le reste de la division euclidenne de $I(x)$ par $g(x)$ le polynome générateur

In [ ]:
class ReedSolomon(ReedSolomon): 
    def encode(self,message,nbErrorBits):
        polynomial = self._galDvtPolynome(nbErrorBits)
        charCodes = list(map(ord,message))+[0]*nbErrorBits
        charCodes[-nbErrorBits:] = self._galPolynomialDivision(charCodes,polynomial)[1]
        return charCodes

In [ ]:
import functools
rs = ReedSolomon()
maxErreurs = 8
message = "salut !"
encoded = rs.encode(message,maxErreurs*2)

print("Bits d'information",encoded[:len(message)])
print("Bits d'encodage",encoded[len(message):])

In [ ]:
functools.reduce(lambda a,b : a + b,map(chr,encoded))

<a id="solomon33"></a>
### 2.3.3.3 Décodage d'un message
<a id="solomon331"></a>
#### 2.3.3.3.1 Générer le polynôme de syndrôme
Nous devons donc générer un polynôme dit de "syndrome" pour voir si le message a été altéré lors de la transmission
En nous appuyant sur ce [pdf](https://infoscience.epfl.ch/record/198834/files/rs.pdf), nous pouvons voir comment un polynôme de syndrôme est créé.
Le polynôme d'erreur $\Delta(x)$ reçu est donc la différence entre le code envoyé $E(x)$ et le code reçu $R(x)$, c.à.d : $$\Delta(x) = E(x) - R(x) = E(x) + R(x)$$

Soit $S(x)$ le polynome de syndrome constitué de coefficients $$S_i = \Delta(\alpha^i) \; \forall i \in [\![1,2t]\!]$$ <br>
<center>On a alors $S_i = E(\alpha^i) + R(\alpha^i)$,<br><br> Or,  $E(\alpha^i) = 0 \; \forall i \in [\![1,2t]\!]$<br><br> car $E(x) = I(x)x^{2t} \times g(x)$<br><br>et $g(\alpha^i) = 0$</center>

Le polynôme de syndrôme est donc l'évaluation de toutes les racines du message reçu $R(x)$. Si le message ne contient pas d'erreur, on aura alors le syndrome n'aura que des coefficients égaux à 0, sinon cela signifie qu'une erreur s'est glissée lors de la transmission du message.

Pour évaluer le polynôme nous utiliserons le [théorème du reste](http://www.mesacc.edu/~scotz47781/mat120/notes/divide_poly/remainder/remainder_thm.html#:~:text=Apply%20the%20Remainder%20Theorem%2C%20which,the%20answer%20is%20the%20remainder), qui dit que : 

<br><center>Si un polynome $f(x)$ est divisé par $(x-c)$ alors le reste de la division euclidienne est $f(c)$
    
**Implémentons donc la fonction générant le polynôme de syndrôme :**

In [ ]:
class ReedSolomon(ReedSolomon): 
    # retourne le reste de la division polynomiale du message par (x-alpha)
    def _galEvaluatePolynomial(self,encoded,alpha):
        return self._galPolynomialDivision(encoded,[1,alpha])[1][0]
    
    def _galGenerateSyndrome(self,encoded,nbErrorBits):
        polynomial = []
        # On évalue chaque racine (alpha^i = antilog[i]), pour i dans [1,nbErrorBits]
        # Et on l'ajoute à la liste
        for i in range(nbErrorBits):
            polynomial.insert(0,self._galEvaluatePolynomial(encoded,self.antilog[i]))
        return polynomial

In [ ]:
rs = ReedSolomon()
rs._galGenerateSyndrome(encoded,16)

In [ ]:
encoded[5] = -1
rs._galGenerateSyndrome(encoded,16)

Notre fonction de génération de syndrome fonctionne ! 
<a id="solomon332"></a>
#### 2.3.3.3.2 Trouver les polynômes de localisation et d'évaluation d'erreur
Pour la suite, nous utiliserons [cet article](https://www.ias.ac.in/article/fulltext/reso/012/04/0037-0051), qui informe sur plusieurs points: 

1. Il existe un polynome de localisation d'erreur $L(x)$ tel que :

$$L(x) = (1-\alpha^{e_1}x)(1-\alpha^{e_2}x)...(1-\alpha^{e_n}x)$$
<center>avec $e_{0...n}$ les positions des erreurs</center>On détermine facilement que les racines de $L(x)$ seront toutes les valeurs de la forme $\alpha^{-e_n}$

Les racines sont donc **l'inverse de la position des erreurs** _(à ne pas oublier comme l'a fait un certain programmeur **anonyme** qui a passé 3h à ne pas comprendre pourquoi son code ne fonctionnait pas)_

2. Il existe cette relation suivante entre $L(x)$, $S(x)$ le syndrome et $x^{2t}$<center>$L(x)S(x) = a(x)+b(x)x^{2t}$ <br> $<=> L(x)S(x) + b(x)x^{2t} = a(x)$</center>

3. $a(x)$ est le polynome d'évaluation d'erreur, il sera noté plus tard $\omega$

On reconnait ici une relation de Bézout, on peut alors chercher les coefficients inconnus $L(x)$ et $b(x)$ ainsi que le PGCD, et polynome d'évaluation $a(x)$ grâce à l'algorithme d'Euclide. La condition d'arrêt est cependant changée : On s'arrête si le degré du reste est infénieur à $t$.

Voici alors le pseudocode de notre algorithme : 

```
DEBUT
fonction euclide_etendu(polyA,polyB,degreMinimum)
    r_precedent = copie_profonde(polyA)
    r_courant   = copie_profonde(polyB)
    
    u_precedent = 1
    u_courant   = 0
    
    v_precedent = 0
    v_courant   = 1
    
    boucle infinie
        quotient, reste = r_precedent / r_courant  ,   r_precedent % r_courant
        
        u_courant,u_precendent = u_precedent + quotient*u_courant, u_courant
        v_courant,v_precendent = v_precedent + quotient*v_courant, v_courant
        
        r_precedent,r_courant = r_courant, reste
        
        si degre(r_courant <= degreMinimum)
            renvoyer r_courant,u_courant,v_courant
FIN 
```

Ainsi que son adaptation en utilisant les formules du champs $GF(2^8)$

In [ ]:
class ReedSolomon(ReedSolomon):
    def _degree(self,arr):
        for i in range(len(arr)):
            if arr[i] != 0:
                return len(arr)-i-1
        return 0
    
    def _galEuclideanAlgorithm(self,a,b,minDeg):
        r0 = a.copy()
        r1 = b.copy()

        u0 = [1]
        u1 = [0]

        v0 = [0]
        v1 = [1]
        while True:
            q,r = self._galPolynomialDivision(r0,r1)
            u1,u0 = self._galPolynomialAddition(u0,self._galMultiplicationPolynomiale(q,u1)),u1
            v1,v0 = self._galPolynomialAddition(v0,self._galMultiplicationPolynomiale(q,v1)),v1
            r0,r1 = r1,r
            if(self._degree(r1)<=minDeg):
                return r1,u1,v1

In [ ]:
rs = ReedSolomon()
a = [12,54,21,240,21,42,15,42,10,41]
b = [1,4,2,1,5,24,15,34,152]

gcd,u,v = rs._galEuclideanAlgorithm(a,b,5)

summation = rs._galPolynomialAddition(rs._galMultiplicationPolynomiale(a,u),rs._galMultiplicationPolynomiale(b,v))

rs._trimArray(summation)
print(gcd,u,v)
assert(summation == gcd)
assert(rs._degree(gcd)<=5)

Notre fonction renvoie donc bien un couple u,v et un pgcd tel que $$au+bv = gcd$$Notre algorithme d'Euclide est donc fonctionnel
<a id="solomon333"></a>
#### 2.3.3.3.3 Trouver les racines du polynôme de localisation d'erreur

Nous devons trouver les racines du polynôme d'erreur car, rappelons la formule de $L(x)$ : 

$$L(x) = (1-\alpha^{e_1}x)(1-\alpha^{e_2}x)...(1-\alpha^{e_n}x)$$
<center>avec $e_{0...n}$ les positions des erreurs</center>
<center>
Si $e = \alpha^i$ est une racine de $L(x)$, on a $(1-\alpha^{p}e) = 0$<br>
$$<=> e = \alpha^{-p} = \alpha^{q-1}\alpha^{-p} = \alpha^{q-p-1}$$  
$$<=> p = q-i-1$$

avec $q$ la longueur du message
</center>

Pour rechercher ces racines, nous utiliserons un algorithme simple, puissant, et fréquemment utilisé lors du décodage des codes cycliques, [l'algorithme de recherche de Chien](https://en.wikipedia.org/wiki/Chien_search).

Voici le déroulement de cet algorithme :

$${\displaystyle {\begin{array}{lllllllllll}\Lambda (\alpha ^{i})&=&\lambda _{0}&+&\lambda _{1}(\alpha ^{i})&+&\lambda _{2}(\alpha ^{i})^{2}&+&\cdots &+&\lambda _{t}(\alpha ^{i})^{t}\\&\triangleq &\gamma _{0,i}&+&\gamma _{1,i}&+&\gamma _{2,i}&+&\cdots &+&\gamma _{t,i}\\\Lambda (\alpha ^{i+1})&=&\lambda _{0}&+&\lambda _{1}(\alpha ^{i+1})&+&\lambda _{2}(\alpha ^{i+1})^{2}&+&\cdots &+&\lambda _{t}(\alpha ^{i+1})^{t}\\&=&\lambda _{0}&+&\lambda _{1}(\alpha ^{i})\,\alpha &+&\lambda _{2}(\alpha ^{i})^{2}\,\alpha ^{2}&+&\cdots &+&\lambda _{t}(\alpha ^{i})^{t}\,\alpha ^{t}\\&=&\gamma _{0,i}&+&\gamma _{1,i}\,\alpha &+&\gamma _{2,i}\,\alpha ^{2}&+&\cdots &+&\gamma _{t,i}\,\alpha ^{t}\\&\triangleq &\gamma _{0,i+1}&+&\gamma _{1,i+1}&+&\gamma _{2,i+1}&+&\cdots &+&\gamma _{t,i+1}\end{array}}}
$$

Si à un moment de l'algorithme : $$\ \sum _{{j=0}}^{t}\gamma _{{j,i}}=0,$$
Alors $$\Lambda (\alpha ^{i})=0$$

c.à.d $\alpha^{i}$ est une racine de $\Lambda$

In [ ]:
class ReedSolomon(ReedSolomon):
    def _galChienSearch(self,locator,msgLen):
        
        alpha = self._galDiv(1,self.antilog[self.gfSize - msgLen])

        chien_array = [0] * len(locator)
        for i in range(len(locator)):
            chien_array[i] = self._galMult(locator[i], self._galPow(alpha, i))
        roots = []
        for i in range(msgLen):
            res = functools.reduce(lambda x,y: x^^y, chien_array)
            for j in range(1, len(chien_array)):
                index = len(chien_array) - j - 1
                chien_array[index] = self._galMult(chien_array[index], self.antilog[j])
            if res == 0:
                roots.append(msgLen - i -1)
        return roots

In [ ]:
rs = ReedSolomon()
maxErreurs = 8
message = "salut !"
encoded = rs.encode(message,(maxErreurs*2))
#Création d'une erreur à la position 1
encoded[1]=100
syndrome = rs._galGenerateSyndrome(encoded,maxErreurs*2)

x = [1]+[0]*(maxErreurs*2)

evaluator,_,locator = rs._galEuclideanAlgorithm(x,syndrome,maxErreurs)

errors = rs._galChienSearch(locator,len(encoded))

assert(len(encoded) - 1 - errors[0] == 1)


Nous avons désormais la position de notre erreur !

Il ne nous reste alors plus qu'à la corriger grâce à notre polynôme d'évaluation d'erreur.

On admet cette formule :
∀i,L(αi)=0,ei=ω(αi)L′(αi)
Avec ω le polynôme évaluateur d'erreur

Corriger une erreur est alors très simple, pour chaque position d'erreur (rappel : une racine a été trouvée), on calcule la division de ω et L′ à l'inverse de la racine.

On multiplie ensuite la valeur trouvée par la racine, et on trouve notre delta correspondant à l'erreur, que l'ont corrigera en "xor-ant" sur le message corrompu.

On trouvera L′ grâce au Schéma de Horner qui permet d'évaluer itérativement une dérivée grâce à sa fonction principale.

In [ ]:
class ReedSolomon(ReedSolomon):
    def horner(self,polynomial,x):
        result = 0
        deriv = 0
        for a in polynomial:
            deriv = self._galMult(x,deriv)^^result
            result = self._galMult(x,result)^^a
        return result,deriv
    
    def _galCorrectErrors(self,message,errors,locator,evaluator):
        msg = message.copy()
        offset = len(message)-1
        realPositions = []
        for error in errors:
            evalIndex = self.antilog[error]
            invEval = self._galDiv(1,evalIndex)
            k_inv = self._galDiv(
                self._galEvaluatePolynomial(evaluator,invEval),
                self.horner(locator,invEval)[1]
                )
            errVal =self._galMult(evalIndex,k_inv)
            realPositions.append(offset-error)
            msg[offset-error] ^^=errVal
        print("Erreurs décodées :",realPositions)
        return msg

<a id="solomon334"></a>
#### 2.3.3.3.4 Compléter la fonction de décodage
Nous avons désormais tout les outils pour compléter notre fonction de décodage ! 

On procède au jeu de légo sans plus tard et on a désormais cette fonction :

In [ ]:
class ReedSolomon(ReedSolomon):
    def decode(self,message,nbErrorBits):
            syndrome = self._galGenerateSyndrome(message,nbErrorBits)
            hasError = False
            for val in syndrome:
                if val > 0:
                    hasError = True
            if not hasError:
                return message[:-nbErrorBits]
            x = [1]+[0]*nbErrorBits
            evaluator,_,locator = self._galEuclideanAlgorithm(x,syndrome,nbErrorBits/2)
            self._trimArray(evaluator)
            self._trimArray(locator)
            print(evaluator,locator)
            errors = self._galChienSearch(locator,len(message))
            if(len(errors)==0):
                print(nbErrorBits/2,'erreurs ou plus, décodage impossible')
                return None
            print(len(errors),"erreurs détectées : ")

            return self._galCorrectErrors(message,errors,locator,evaluator)

In [ ]:
rs = ReedSolomon()
maxErreurs = 8
message = "salut !"
encoded = rs.encode(message,maxErreurs*2)
correct = encoded.copy()
encoded[1]=100
encoded[2]=100
print("Encodé avec erreurs:",encoded)
decoded = rs.decode(encoded,maxErreurs*2)
print("Message corrigé : ", decoded)
print("Reussi ? :", correct == decoded)


<a id="solomon4"></a>
### 2.3.4.Code complet : 


In [ ]:
import functools
class ReedSolomon(ReedSolomon):
    # On travaille dans un corps de Galois de valeur 2^8 = 256
    gfSize = 256 
    # http://icaiit.org/proceedings/2nd_ICAIIT/7.pdf
    # Le polynome générateur de la forme x^8 + x^4 + x^3 + x^2 + 1 = 100011101 = 285
    genPoly = 285 
    log = [0]*gfSize
    antilog = [0]*gfSize
    
    def _trimArray(self,arr):
        if len(arr)>0:
            while(len(arr)>0 and arr[0]==0): arr.pop(0)
                
    def _genLogAntilogArrays(self):
        self.antilog[0] = 1
        #log[0] n'existe pas
        self.log[0] = None
        #antilog[0] = antilog[255] car on retourne au début du cycle
        self.antilog[255] = 1
        for i in range(1,255):
            #On multiplie par 2 (équivalent à déplacer tout les bits d'une position vers la gauche, et plus parlant comme on utilise un code cyclique)
            self.antilog[i] = self.antilog[i-1] << 1 
            if self.antilog[i] >= self.gfSize:
                self.antilog[i] = self.antilog[i] ^^ self.genPoly #Operateur XOR sous sagemath
            self.log[self.antilog[i]] = i
            
    def __init__(self):
        self._genLogAntilogArrays()
        
    def _galMult(self,x,y):
            if ((x == 0) or (y == 0)):
                val = 0
            else:
                val = self.antilog[(self.log[x] + self.log[y])%255]
            return val
        
    def _galDiv(self, x, y):
        val = 0
        if (y == 0):
            raise ZeroDivisionError()
        if (x == 0):
            val = 0
        else:
            val = self.antilog[(self.log[x] - self.log[y] + 255)%255]
        return val
    
    def _galPow(self,x, p):
        if x == 0:
            return 0
        return self.antilog[(self.log[x] * p) % 255] #On reste congru modulo 256 (donc indice 255)
    
    def _galPolynomialAddition(self, a, b):
            polSum = [0] * max(len(a), len(b))
            for index in range(0, len(a)):
                polSum[index + len(polSum) - len(a)] = a[index]
            for index in range(0, len(b)):
                polSum[index + len(polSum) - len(b)] ^^= b[index]
            return (polSum)
        
    def _galMultiplicationPolynomiale(self, x,y):
            result = [0]*(len(x)+len(y)-1)
            for i in range(len(x)):
                for j in range(len(y)):
                    result[i+j] ^^= self._galMult(x[i],y[j])
            return result
        
    def _galPolynomialDivision(self,dividend, divisor):
        sor = divisor.copy()
        end = dividend.copy()
        self._trimArray(end)
        self._trimArray(sor)
        normalizer = sor[0]
        if(len(end)==1 or len(sor)==1):
            end.append(0)
            sor.append(0)
        out = end.copy()
        for i in range(len(end)-(len(sor)-1)):
            out[i] =self._galDiv(out[i],normalizer)
            coef = out[i]
            if coef != 0: 
                for j in range(1, len(sor)): 
                    out[i + j] ^^= self._galMult(sor[j], coef)
        separator = -(len(sor)-1)
        return out[:separator], out[separator:]
    
    def _galDvtPolynome(self,nbErrorBits):
        # On commence à multiplier à la valeur 1
        result = [1]
        # On multiplie autant de racines que de bits nécessaires pour trouver les erreurs => (x + 1)*(x + 2)*(x + 4)*....*(x +a**errorBits)
        for index in range(nbErrorBits):
            # Racine du polynome (x + a**index)
            root = [1, self.antilog[index]]
            result = self._galMultiplicationPolynomiale(result, root)
        return result
    
    def encode(self,message,nbErrorBits):
        polynomial = self._galDvtPolynome(nbErrorBits)
        charCodes = list(map(ord,message))+[0]*nbErrorBits
        charCodes[-nbErrorBits:] = self._galPolynomialDivision(charCodes,polynomial)[1]
        return charCodes
    
    # retourne le reste de la division polynomiale du message par (x-alpha)
    def _galEvaluatePolynomial(self,encoded,alpha):
        return self._galPolynomialDivision(encoded,[1,alpha])[1][0]
    
    def _galGenerateSyndrome(self,encoded,nbErrorBits):
        polynomial = []
        # On évalue chaque racine (alpha^i = antilog[i]), pour i dans [1,nbErrorBits]
        # Et on l'ajoute à la liste
        for i in range(nbErrorBits):
            polynomial.insert(0,self._galEvaluatePolynomial(encoded,self.antilog[i]))
        return polynomial
    
    def _degree(self,arr):
        for i in range(len(arr)):
            if arr[i] != 0:
                return len(arr)-i-1
        return 0
    
    def _galEuclideanAlgorithm(self,a,b,minDeg):
        r0 = a.copy()
        r1 = b.copy()

        u0 = [1]
        u1 = [0]

        v0 = [0]
        v1 = [1]
        while True:
            q,r = self._galPolynomialDivision(r0,r1)
            u1,u0 = self._galPolynomialAddition(u0,self._galMultiplicationPolynomiale(q,u1)),u1
            v1,v0 = self._galPolynomialAddition(v0,self._galMultiplicationPolynomiale(q,v1)),v1
            r0,r1 = r1,r
            if(self._degree(r1)<=minDeg):
                return r1,u1,v1
            
    def _galChienSearch(self,locator,msgLen):
        
        alpha = self._galDiv(1,self.antilog[self.gfSize - msgLen])

        chien_array = [0] * len(locator)
        for i in range(len(locator)):
            chien_array[i] = self._galMult(locator[i], self._galPow(alpha, i))
        roots = []
        for i in range(msgLen):
            res = functools.reduce(lambda x,y: x^^y, chien_array)
            for j in range(1, len(chien_array)):
                index = len(chien_array) - j - 1
                chien_array[index] = self._galMult(chien_array[index], self.antilog[j])
            if res == 0:
                roots.append(msgLen - i -1)
        return roots
    
    def horner(self,polynomial,x):
        result = 0
        deriv = 0
        for a in polynomial:
            deriv = self._galMult(x,deriv)^^result
            result = self._galMult(x,result)^^a
        return result,deriv
    
    def _galCorrectErrors(self,message,errors,locator,evaluator):
        msg = message.copy()
        offset = len(message)-1
        realPositions = []
        for error in errors:
            evalIndex = self.antilog[error]
            invEval = self._galDiv(1,evalIndex)
            k_inv = self._galDiv(
                self._galEvaluatePolynomial(evaluator,invEval),
                self.horner(locator,invEval)[1]
                )
            errVal =self._galMult(evalIndex,k_inv)
            realPositions.append(offset-error)
            msg[offset-error] ^^=errVal
        print("Erreurs décodées :",realPositions)
        return msg
    
    def decode(self,message,nbErrorBits):
            syndrome = self._galGenerateSyndrome(message,nbErrorBits)
            hasError = False
            for val in syndrome:
                if val > 0:
                    hasError = True
            if not hasError:
                return message[:-nbErrorBits]
            x = [1]+[0]*nbErrorBits
            evaluator,_,locator = self._galEuclideanAlgorithm(x,syndrome,nbErrorBits/2)
            self._trimArray(evaluator)
            self._trimArray(locator)
            print(evaluator,locator)
            errors = self._galChienSearch(locator,len(message))
            if(len(errors)==0):
                print(nbErrorBits/2,'erreurs ou plus, décodage impossible')
                return None
            print(len(errors),"erreurs détectées : ")

            return self._galCorrectErrors(message,errors,locator,evaluator)
        
rs = ReedSolomon()
maxErreurs = 8
message = "salut !"
encoded = rs.encode(message,maxErreurs*2)
correct = encoded.copy()
encoded[1]=100
encoded[2]=100
print("Encodé avec erreurs:",encoded)
decoded = rs.decode(encoded,maxErreurs*2)
print("Message corrigé : ", decoded)
print("Reussi ? :", correct == decoded)

### 2.3.5 Amélioration possible

Actuellement notre code peut corriger jusqu'à $t$ erreurs. Cependant les codes de Hamming ont la possibilité de corriger jusqu'à $2t$ effaçages. Les effaçages sont les erreurs que l'on peut connaître d'office (par exemple une valeur n'appartenant pas à une borne donnée). Il est alors possible de corriger les erreurs ainsi que les effaçages en combinant un polynôme de syndrome avec un polynôme d'effaçages, contiendra simplement les positions des données effacées. Il faudra cependant un peu plus de calculs, car il faudra alors utiliser [l'interpolation de Lagrange](https://en.wikipedia.org/wiki/Lagrange_polynomial) pour qui sera utilisée dans l'[algorithme de Forney](https://en.wikipedia.org/wiki/Forney_algorithm), et l'[algorithme de Berklekamp-Massey](https://en.wikipedia.org/wiki/Berlekamp%E2%80%93Massey_algorithm). 

Ces algorithmes étant plus exigeants en matières de connaissances mathématiques, nous n'avons pas réussi à les implémenter. Il existe cependant une implémentation de ces algorithmes sur [ce site internet](https://rextester.com/ZMBYT68318) qui arrive à l'implémenter bien que possédant tout de même quelques erreurs sur certaines valeurs.

<a id="biblio"></a>
# 4. Bibliographie 

https://fr.wikipedia.org/wiki/Code_de_Reed-Solomon

https://www.ias.ac.in/article/fulltext/reso/012/04/0037-0051

https://infoscience.epfl.ch/record/198834/files/rs.pdf

http://www.lix.polytechnique.fr/Labo/Daniel.Augot/introcyclic-springer.pdf

https://en.wikipedia.org/wiki/Cyclic_code

http://icaiit.org/proceedings/2nd_ICAIIT/7.pdf